In [ ]:
import os
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
from sklearn.ensemble import GradientBoostingRegressor
import matplotlib.pyplot as plt

# Define experimental parameters
Temperature = 293.15  # Kelvin
viscosity = 14e-3  # Pa*s
param = 0.88  # Correction factor for the Gaussian beam profile
w = 10e-6  # Radius of the bleaching spot (m)
kb = 1.38e-23  # Boltzmann constant (J/K)
timestep = 0.2  # Imaging timestep (s)

# Number of experimental runs
num_runs = 10


In [ ]:
# Load the provided Excel file
file_path = r"E:\SodiumSilicates_FRAP\CW6145\CW6145_50_20uM\Processed\RawFRAPCurves.xlsx"
df = pd.read_excel(file_path)
print(df.head())

# Define the exponential models
def exp_model_single(t, b0, b1, tau1):
    return b0 + b1 * (1 - np.exp(-t / tau1))

def exp_model_double(t, b0, b1, tau1, b2, tau2):
    return b0 + b1 * (1 - np.exp(-t / tau1)) + b2 * (1 - np.exp(-t / tau2))


In [ ]:
# Extract frames and runs
Frame = df['Frame'].values
runs = [df[f'Run_{i}'].values for i in range(1, num_runs + 1)]

# Initialize arrays to store results for all runs
tau_single_values = np.zeros(num_runs)
A_single_values = np.zeros(num_runs)
tau_double_values = np.zeros((num_runs, 2))
A_double_values = np.zeros((num_runs, 2))

for run_idx in range(num_runs):
    # Use data from frame 50 onwards
    intensity = runs[run_idx][50:]
    time = Frame[50:]
    
    # Remove NaN values
    valid_mask = ~np.isnan(intensity)
    intensity = intensity[valid_mask]
    time = time[valid_mask]

    # Gradient Boosting Machine preprocessing
    gbm = GradientBoostingRegressor(n_estimators=200, learning_rate=0.1, max_depth=3)
    gbm.fit(time.reshape(-1, 1), intensity)
    predicted_intensity = gbm.predict(time.reshape(-1, 1))

    # Single exponential model fitting
    beta_single, _ = curve_fit(exp_model_single, time, predicted_intensity, 
                               p0=[min(predicted_intensity), 
                                   max(predicted_intensity) - min(predicted_intensity), 
                                   100])
    
    # Double exponential model fitting with bounds
    beta_double, _ = curve_fit(exp_model_double, time, predicted_intensity, 
                           p0=[min(predicted_intensity), 
                               (max(predicted_intensity) - min(predicted_intensity)) / 3, 
                               20, 
                               (max(predicted_intensity) - min(predicted_intensity)) / 3, 
                               100], 
                           bounds=(0, np.inf),  # Only positive values allowed
                           maxfev=5000)

    # Extract tau and A values
    tau_single_values[run_idx] = beta_single[2]
    A_single_values[run_idx] = beta_single[1]

    tau_double = np.sort([beta_double[2], beta_double[4]])
    A_double = [beta_double[1], beta_double[3]]
    A_double = np.array(A_double)[np.argsort([beta_double[2], beta_double[4]])]

    tau_double_values[run_idx, :] = tau_double
    A_double_values[run_idx, :] = A_double


In [ ]:
# Calculate diffusion coefficients
ln2 = np.log(2)

# Single Exponential
T_single = tau_single_values * ln2 * timestep
D_single = param * ((w ** 2) / (4 * T_single))

# Double Exponential
T_double1 = tau_double_values[:, 0] * ln2 * timestep
T_double2 = tau_double_values[:, 1] * ln2 * timestep
D_double1 = param * (w ** 2) / (4 * T_double1)
D_double2 = param * (w ** 2) / (4 * T_double2)

# Display results
print("Diffusion Coefficients for Single Exponential:")
print(D_single)
print("\nDiffusion Coefficients for Double Exponential - Tau1:")
print(D_double1)
print("\nDiffusion Coefficients for Double Exponential - Tau2:")
print(D_double2)


In [ ]:
# Calculate hydrodynamic radii
R_single = (kb * Temperature) / (6 * np.pi * viscosity * D_single) / 1e-9
R_double1 = (kb * Temperature) / (6 * np.pi * viscosity * D_double1) / 1e-9
R_double2 = (kb * Temperature) / (6 * np.pi * viscosity * D_double2) / 1e-9

# Display results
print("\nHydrodynamic Radii for Single Exponential:")
print(R_single)
print("\nHydrodynamic Radii for Double Exponential - Tau1:")
print(R_double1)
print("\nHydrodynamic Radii for Double Exponential - Tau2:")
print(R_double2)


In [ ]:
# Create DataFrames for the results
# Fitting Results
fitting_results = pd.DataFrame({
    'Run': np.arange(1, num_runs + 1),
    'Tau_Single': tau_single_values,
    'A_Single': A_single_values,
    'Tau_Double1': tau_double_values[:, 0],
    'Tau_Double2': tau_double_values[:, 1],
    'A_Double1': A_double_values[:, 0],
    'A_Double2': A_double_values[:, 1]
})

# Size Calculations
size_results = pd.DataFrame({
    'Run': np.arange(1, num_runs + 1),
    'D_Single': D_single,
    'R_Single': R_single,
    'D_Double1': D_double1,
    'R_Double1': R_double1,
    'D_Double2': D_double2,
    'R_Double2': R_double2
})

# Save to Excel
output_dir = os.path.dirname(file_path)
fitting_file_name = "Fitting_Results_GBM.xlsx"
size_file_name = "Size_Calculations_GBM.xlsx"

fitting_file_path = os.path.join(output_dir, fitting_file_name)
size_file_path = os.path.join(output_dir, size_file_name)

fitting_results.to_excel(fitting_file_path, index=False)
size_results.to_excel(size_file_path, index=False)

print("\nResults saved to:")
print(f"1. Fitting Results: {fitting_file_path}")
print(f"2. Size Calculations: {size_file_path}")

In [ ]:

# Plot for Single Exponential
plt.figure(figsize=(10, 6))
plt.plot(np.arange(1, num_runs + 1), R_single, 'o-', label='Single Exponential', color='blue')
plt.xlabel('Run Number')
plt.ylabel('Hydrodynamic Radius (nm)')
plt.title('Hydrodynamic Radius vs. Run Number (Single Exponential)')
plt.grid(True)
plt.legend()
plt.show()

# Plot for Double Exponential
plt.figure(figsize=(10, 6))
plt.plot(np.arange(1, num_runs + 1), R_double1, 'o-', label='Double Exponential Tau1', color='green')
plt.plot(np.arange(1, num_runs + 1), R_double2, 's-', label='Double Exponential Tau2', color='red')
plt.xlabel('Run Number')
plt.ylabel('Hydrodynamic Radius (nm)')
plt.title('Hydrodynamic Radius vs. Run Number (Double Exponential)')
plt.grid(True)
plt.legend()
plt.show()
